In [63]:
from utils import generate_graph_from_nx_graph, graph_preprocessing
import json
import networkx as nx
import pprint
import matplotlib.pyplot as plt

dataset_name = '1138_bus'

print(dataset_name)

DATASET_PATH = f'lib/egraph-rs/js/dataset/{dataset_name}.json'

EDGE_WEIGHT = 30


with open(DATASET_PATH) as f:
    graph_data = json.load(f)
nx_graph = graph_preprocessing(nx.node_link_graph(graph_data), EDGE_WEIGHT)
graph, indices = generate_graph_from_nx_graph(nx_graph)
all_shortest_paths = dict(nx.all_pairs_dijkstra_path_length(nx_graph))


1138_bus


KeyboardInterrupt: 

In [ ]:
from utils import draw_graph

params = {
    "edge_length": 56,
    "number_of_pivots": 917,
    "number_of_iterations": 155,
    "eps": 0.8459816384096752
}
pos = draw_graph(graph, indices, params)


In [ ]:
import time
from utils import edge_crossing_finder
from quality_metrics import angular_resolution, aspect_ratio, crossing_angle, crossing_number, gabriel_graph_property, ideal_edge_length, node_resolution, shape_based_metrics, stress
import numpy as np


def calc_qs(nx_graph, pos, all_shortest_paths, qnames, edge_weight=1):
    start = time.time()
    result = {}
    edge_crossing = None
    if 'crossing_angle' in qnames or 'crossing_number' in qnames:
        edge_crossing = edge_crossing_finder(nx_graph, pos)

    for qname in qnames:
        if qname == 'angular_resolution':
            result[qname] = angular_resolution.quality(nx_graph, pos)
        elif qname == 'aspect_ratio':
            result[qname] = aspect_ratio.quality(nx_graph, pos)
        elif qname == 'crossing_angle':
            result[qname] = crossing_angle.quality(
                nx_graph, pos, edge_crossing)
        elif qname == 'crossing_number':
            result[qname] = crossing_number.quality(
                nx_graph, pos, edge_crossing)
        elif qname == 'gabriel_graph_property':
            result[qname] = gabriel_graph_property.quality(nx_graph, pos)
        elif qname == 'ideal_edge_length':
            result[qname] = ideal_edge_length.quality(
                nx_graph, pos, all_shortest_paths)
        elif qname == 'node_resolution':
            result[qname] = node_resolution.quality(pos)
        elif qname == 'shape_based_metrics':
            result[qname] = shape_based_metrics.quality(
                nx_graph, pos, edge_weight)
        elif qname == 'stress':
            result[qname] = stress.quality(nx_graph, pos, all_shortest_paths)

    print(time.time() - start)
    return result

In [ ]:
calc_qs(nx_graph, pos, all_shortest_paths, [
        'angular_resolution',
        'aspect_ratio',
        'crossing_angle',
        'crossing_number',
        'gabriel_graph_property',
        'ideal_edge_length',
        'node_resolution',
        'shape_based_metrics',
        'stress'
    ], edge_weight=EDGE_WEIGHT)

In [ ]:
nx.draw(nx_graph, pos, node_size=1)
plt.show()

In [ ]:
target_opt_path = 'data/optimization/bull/q=all_n_trials=100_d=2022-11-10_14-17-37.json'

# targetとなるparamsデータ取得
with open(target_opt_path) as f:
    opt_data = json.load(f)

opt_params = []
for best_num in opt_data['best_trial_numbers']:
    trial = opt_data['trials'][f'{best_num}']
    p = trial['params']
    print(trial)

In [ ]:
ds = [
'3elt',
'1138_bus',
'bull',
'chvatal',
'cubical',
'davis_southern_women',
'desargues',
'diamond',
'dodecahedral',
'dwt_1005',
'dwt_2680',
'florentine_families',
'frucht',
'heawood',
'hoffman_singleton',
'house_x',
'house',
'icosahedral',
'karate_club',
'krackhardt_kite',
'les_miserables',
'moebius_kantor',
'octahedral',
'pappus',
'petersen',
'poli',
'qh882',
'sedgewick_maze',
'tutte',
'USpowerGrid']

In [ ]:
rs = []

for dataset_name in ds:
    DATASET_PATH = f'lib/egraph-rs/js/dataset/{dataset_name}.json'

    with open(DATASET_PATH) as f:
        graph_data = json.load(f)
    nx_graph = graph_preprocessing(nx.node_link_graph(graph_data), EDGE_WEIGHT)

    # print(dataset_name, len(nx_graph.nodes), len(nx_graph.edges))
    rs.append((dataset_name, len(nx_graph.nodes), len(nx_graph.edges)))

In [85]:
rs = sorted(rs, key=lambda x: x[1])
pprint.pprint(rs)

[('diamond', 4, 5),
 ('bull', 5, 5),
 ('house_x', 5, 8),
 ('house', 5, 6),
 ('octahedral', 6, 12),
 ('cubical', 8, 12),
 ('sedgewick_maze', 8, 10),
 ('krackhardt_kite', 10, 18),
 ('petersen', 10, 15),
 ('chvatal', 12, 24),
 ('frucht', 12, 18),
 ('icosahedral', 12, 30),
 ('heawood', 14, 21),
 ('florentine_families', 15, 20),
 ('moebius_kantor', 16, 24),
 ('pappus', 18, 27),
 ('desargues', 20, 30),
 ('dodecahedral', 20, 30),
 ('davis_southern_women', 32, 89),
 ('karate_club', 34, 78),
 ('tutte', 46, 69),
 ('hoffman_singleton', 50, 175),
 ('les_miserables', 77, 254),
 ('qh882', 882, 1533),
 ('dwt_1005', 1005, 3808),
 ('1138_bus', 1138, 1458),
 ('poli', 2343, 2667),
 ('dwt_2680', 2680, 11173),
 ('3elt', 4720, 13722),
 ('USpowerGrid', 4941, 6594)]


In [84]:
target_pareto_qs = 'angular_resolution,aspect_ratio'

all_qnames = [
    'angular_resolution',
    'aspect_ratio',
    'crossing_angle',
    'crossing_number',
    'gabriel_graph_property',
    'ideal_edge_length',
    'node_resolution',
    'shape_based_metrics',
    'stress'
]

target_pareto_qnames = [
    qname for qname in all_qnames] if target_pareto_qs == 'all' else target_pareto_qs.split(',')


# targetとなる最適化済みparamsデータ取得
with open('data/optimization/USpowerGrid/q=all_n_trials=100_d=2022-11-10_14-17-37.json') as f:
    opt_data = json.load(f)


inf = float('inf')
max_pareto_params = {
    'angular_resolution': None,
    'aspect_ratio': None,
    'crossing_angle': None,
    'crossing_number': None,
    'gabriel_graph_property': None,
    'ideal_edge_length': None,
    'node_resolution': None,
    'shape_based_metrics': None,
    'stress': None
}

qmap = {
    'angular_resolution': angular_resolution,
    'aspect_ratio': aspect_ratio,
    'crossing_angle': crossing_angle,
    'crossing_number': crossing_number,
    'gabriel_graph_property': gabriel_graph_property,
    'ideal_edge_length': ideal_edge_length,
    'node_resolution': node_resolution,
    'shape_based_metrics': shape_based_metrics,
    'stress': stress
}
m = {
        'angular_resolution': -inf,
        'aspect_ratio': -inf,
        'crossing_angle':  inf,
        'crossing_number':  inf,
        'gabriel_graph_property': -inf,
        'ideal_edge_length':  inf,
        'node_resolution': -inf,
        'shape_based_metrics': -inf,
        'stress':  inf,
    }

opt_params = []
seed = 0
for best_num in opt_data['best_trial_numbers']:
    trial = opt_data['trials'][f'{best_num}']
    qs = trial['quality']
    pos = trial['pos']
    params = trial['params']

    for tpq in all_qnames:
        q = qs[tpq]
        if qmap[tpq].direction == 'maximize':
            if m[tpq] < q:
                m[tpq] = q
        elif qmap[tpq].direction == 'minimize':
            if q < m[tpq]:
                m[tpq] = q


    for tpq in target_pareto_qnames:
        q = qs[tpq]
        if max_pareto_params[tpq] is None:
            max_pareto_params[tpq] = {
                    'trial_n': best_num,'params': params, 'quality': qs, 'seed': seed, 'pos': pos
            }
        if qmap[tpq].direction == 'maximize':
            if max_pareto_params[tpq]['quality'][tpq] < q:
                max_pareto_params[tpq] = {
                    'trial_n': best_num,'params': params, 'quality': qs, 'seed': seed, 'pos': pos
                }
        elif qmap[tpq].direction == 'minimize':
            if q < max_pareto_params[tpq]['quality'][tpq]:
                max_pareto_params[tpq] = {
                    'trial_n': best_num,'params': params, 'quality': qs, 'seed': seed, 'pos': pos
                }

pprint.pprint(m)
{
        "angular_resolution": 3.4290159314019115e-5,
        "aspect_ratio": 0.8666761004741643,
        "crossing_angle": 0.9986419546337182,
        "crossing_number": 14474,
        "gabriel_graph_property": 0.009093854475820016,
        "ideal_edge_length": 2915.0515187151223,
        "node_resolution": 0.0017676985543427887,
        "shape_based_metrics": 0.05016736712298027,
        "stress": 357446.10639993486
}
# for i in max_pareto_params:
#     print(max_pareto_params[i]['trial_n'])

{'angular_resolution': 3.4290159314019115e-05,
 'aspect_ratio': 0.8666761004741643,
 'crossing_angle': inf,
 'crossing_number': 14474,
 'gabriel_graph_property': 0.009093854475820016,
 'ideal_edge_length': 2915.0515187151223,
 'node_resolution': 0.0017676985543427887,
 'shape_based_metrics': 0.05016736712298027,
 'stress': 357446.10639993486}


{'angular_resolution': 3.4290159314019115e-05,
 'aspect_ratio': 0.8666761004741643,
 'crossing_angle': 0.9986419546337182,
 'crossing_number': 14474,
 'gabriel_graph_property': 0.009093854475820016,
 'ideal_edge_length': 2915.0515187151223,
 'node_resolution': 0.0017676985543427887,
 'shape_based_metrics': 0.05016736712298027,
 'stress': 357446.10639993486}